<a href="https://colab.research.google.com/github/ringerH/Assignment/blob/main/4_bit_llama_2_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [2]:
!pip install -q -U bitsandbytes
!pip install transformers==4.31
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB

**Core Architecture**

**1.Quantization: Storing weights of neural networks in low-precision number format to accommodate training in less-memory environments efficiently.
here the weights are stored in 4-bit precision in normalized float format and then double quantization to ensure training efficiency.**

**2.Low-Rank Adaptation: Here, the self-attention projections are factorized into two low ranked matrices which facilitate mathematically efficient calculation over linear transformations. The projections Query, Key, Value, Output are assumed low ranked matrices.**
**Q=A.B**

**3.Mixed-Precision: Half-precision(16-bit) and adam 8-bit optimizer used to mitigate accuracy and computational efficiency over 32-bit**

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "Trelis/Llama-2-7b-chat-hf-sharded-bf16"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/719 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

...

pytorch_model-00005-of-00007.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

pytorch_model-00006-of-00007.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

pytorch_model-00007-of-00007.bin:   0%|          | 0.00/1.66G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/197 [00:00<?, ?B/s]

In [4]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [5]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainables: {trainable_params} || total: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [6]:
from peft import LoraConfig, get_peft_model
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["self_attn.q_proj", "self_attn.k_proj", "self_attn.v_proj", "self_attn.o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)
print_trainable_parameters(model)

trainables: 8388608 || total: 3508801536 || trainable%: 0.23907331075678143


In [7]:
from datasets import load_dataset

data = load_dataset("nisaar/LLAMA2_Legal_Dataset_4.4k_Instructions")
data = data.map(lambda samples: tokenizer(samples["instruction"]), batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/4394 [00:00<?, ? examples/s]

In [10]:
import transformers

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=20,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,4.033300
2,3.814700
3,3.357600
4,3.311000
5,3.265200
6,2.651500
7,2.448300
8,2.228800
9,2.650500
10,2.145800


TrainOutput(global_step=20, training_loss=2.555773341655731, metrics={'train_runtime': 104.8751, 'train_samples_per_second': 0.763, 'train_steps_per_second': 0.191, 'total_flos': 26711085170688.0, 'train_loss': 2.555773341655731, 'epoch': 0.02})

In [12]:
import plotly.graph_objects as go


steps = [i for i in range(1, 21)]
training_loss = [4.033300, 3.814700, 3.357600, 3.311000, 3.265200, 2.651500, 2.448300, 2.228800, 2.650500, 2.145800,
                 1.954100, 2.418300, 2.222900, 2.141500, 1.569400, 2.258800, 2.302800, 1.715200, 2.181900, 2.443700]


additional_steps = [i for i in range(1, 31)]
additional_training_loss = [0.414500, 0.444000, 0.357600, 0.281700, 0.381800, 0.323000, 0.139800, 0.872500, 1.699000, 0.457000,
                            0.533800, 0.510700, 1.013200, 0.985700, 0.269100, 0.602100, 1.239300, 0.643400, 1.668200, 2.174400,
                            0.839600, 0.788300, 0.929700, 0.867500, 1.019900, 0.332600, 0.604300, 0.584300, 1.343900, 0.820200]

avg_training_loss_original = sum(training_loss) / len(training_loss)
avg_training_loss_additional = sum(additional_training_loss) / len(additional_training_loss)

print(f"Average Training Loss (20 steps): {avg_training_loss_original:.4f}")

print(f"Average Training Loss (30 steps): {avg_training_loss_additional:.4f}")


trace_original = go.Scatter(x=steps, y=training_loss, mode='lines+markers', name='Training Loss (Original)')
trace_additional = go.Scatter(x=additional_steps, y=additional_training_loss, mode='lines+markers', name='Training Loss (Additional)')

fig = go.Figure()


fig.add_trace(trace_original)
fig.add_trace(trace_additional)

fig.update_layout(title='Training Loss Over Steps', xaxis=dict(title='Step'), yaxis=dict(title='Training Loss'))

fig.show()


Average Training Loss (20 steps): 2.5558
Average Training Loss (30 steps): 0.7714


In [ ]:
from transformers import TextStreamer
model.config.use_cache = True
model.eval()

In [17]:
import time

def stream_and_measure_time(model, tokenizer, user_prompt, max_new_tokens=500):
    runtime_flag = "cuda:0"
    system_prompt = 'You are quantized'

    b_inst, e_inst = "[INST]", "[/INST]"
    b_sys, e_sys = "<<SYS>>\n", "\n<</SYS>>\n\n"

    prompt = f"{b_inst} {b_sys}{system_prompt.strip()}{e_sys}{user_prompt.strip()} {e_inst}\n\n"

    inputs = tokenizer([prompt], return_tensors="pt").to(runtime_flag)

    streamer = TextStreamer(tokenizer)

    start_time = time.time()
    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=max_new_tokens)
    end_time = time.time()

    inference_time = end_time - start_time

    return inference_time


In [19]:
inference_time = stream_and_measure_time(model, tokenizer, 'Explain the Section 13(B) in 2 lines.')
print(f"Inference Time: {inference_time:.4f} seconds")
#after varous sets of training over same number of steps results in large inference time

<s> [INST] <<SYS>>
You are quantized
<</SYS>>

Explain the Section 13(B) in 2 lines. [/INST]

Section 13(B) of the provided Act states that the provisions of the Act shall have effect notwithstanding anything inconsistent therewith contained in any other law for the time being in force.
This means that the provisions of the Act supersede any conflicting provisions in other laws, and the Act's provisions shall have precedence in any matter related to the subject matter of the Act.</s>
Inference Time: 17.8763 seconds


In [40]:
base_model_name = model_id.split("/")[-1]

adapter_model = f"hillol7/{base_model_name}-fine-tuned-adapters"
new_model = f"hillol7/{base_model_name}-fine-tuned"

In [41]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [42]:
model.save_pretrained(adapter_model, push_to_hub=True, use_auth_token=True)
model.push_to_hub(adapter_model, use_auth_token=True)

adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hillol7/Llama-2-7b-chat-hf-sharded-bf16-fine-tuned-adapters/commit/2734734ed013669cf44381dacaea3a5df2b76fda', commit_message='Upload model', commit_description='', oid='2734734ed013669cf44381dacaea3a5df2b76fda', pr_url=None, pr_revision=None, pr_num=None)